In [ ]:
import os

HUGGINGFACEHUB_API_TOKEN = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ['OPENAI_API_KEY'] = ''

Document loading

In [2]:
import csv
from langchain.schema import Document

csv.field_size_limit(10 * 1024 * 1024)

131072

In [3]:
DATA_DIR = "../data/"
news_data = []
with open(f"{DATA_DIR}khas_bank_news.csv", newline='', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile, fieldnames=["Title", "Link", "Date", "Content"]) 
    next(reader) # Skip the header row 
    for row in reader: 
        # Extract relevant information 
        title = row["Title"] 
        link = row["Link"] 
        date = row["Date"] 
        content = title + "\\n\\n" + row["Content"] + "\\n" + date 
        
        # Create metadata dictionary 
        metadata = { 
            "type": "npa",
            "title": title, 
            "source": link, 
            "date": date, 
        } 
        # Create a Langchain Document 
        document = Document(page_content=content, metadata=metadata) 
        news_data.append(document)

product_data = []
with open(f"{DATA_DIR}khas_bank_products.csv", newline='', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile, fieldnames=["Page ID", "Main Text", "Side Menu Text", "link"]) 
    next(reader) # Skip the header row 
    for row in reader: 
        # Extract relevant information 
        title = row["Page ID"] 
        link = row["link"] 
        content = row["Side Menu Text"] + "\\n\\n" + row["Main Text"]
        
        # Create metadata dictionary 
        metadata = {
            "type": "pro", 
            "title": title, 
            "source": link
        } 
        # Create a Langchain Document 
        document = Document(page_content=content, metadata=metadata) 
        product_data.append(document)

pages_data = []
with open(f"{DATA_DIR}khas_bank_pages.csv", newline='', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile, fieldnames=["Title Text", "Main Text", "link"]) 
    next(reader) # Skip the header row 
    for row in reader: 
        # Extract relevant information 
        title = row["Title Text"] 
        link = row["link"] 
        content = row["Title Text"] + "\\n\\n" + row["Main Text"]
        
        # Create metadata dictionary 
        metadata = {
            "type": "npa", 
            "title": title, 
            "source": link
        } 
        # Create a Langchain Document 
        document = Document(page_content=content, metadata=metadata) 
        pages_data.append(document)

branch_data = []
with open(f"{DATA_DIR}khas_bank_branches.csv", newline='', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile, fieldnames=["name","name_en","type","address","address_en","timetable","timetable_en","phone","mon","tue","wed","thu","fri","sat","sun","open_time","close_time"]) 
    next(reader) # Skip the header row 
    for row in reader: 
        # Extract relevant information 
        title = row["name"]
        type_suffix = "салбар" if row["type"] == "branch" else "АТМ"

        content = ( 
            f"{row['name']} {type_suffix} \\n\\n" 
            f"Салбар: {row['address']} \\n" 
            f"Цагийн хуваарь: {row['timetable']} \\n" 
            f"Утас: {row['phone']} \\n" 
        )
        
        # Create metadata dictionary 
        metadata = {
            "type": "bra", 
            "source": title
        } 
        # Create a Langchain Document 
        document = Document(page_content=content, metadata=metadata) 
        branch_data.append(document)

all_data = news_data + product_data + pages_data + branch_data
print(len(news_data), len(product_data), len(pages_data), len(branch_data))
print(f"Нийт хуудасны тоо: {len(all_data)}")
print(all_data[0])

664 71 50 418
Нийт хуудасны тоо: 1203
page_content='ХасБанк 2024 оны III улиралд 43.4 тэрбум төгрөгийн татварын дараах цэвэр ашигтай ажиллалаа\n\nХасБанк 2024 оны III улиралд 43.4 тэрбум төгрөгийн татварын дараах цэвэр ашигтай ажиллалааХасБанк 2024 оны III улиралд 43.4 тэрбум төгрөгийн (өмнөх улирлаас +0.4%) татварын дараах цэвэр ашигтай ажиллалаа. Ингэснээр жилийн өссөн дүнгээр татварын дараах цэвэр ашиг 128.7 тэрбум төгрөгт (өмнөх жилийн мөн үеэс +26.7%) хүрч, өөрийн хөрөнгийн өгөөж (annualized ROE) 26.2%,нэгж хувьцаанд ногдох цэвэр ашиг (EPS) 122.3 төгрөгтхүрлээ2024 оны III улирлын санхүүгийн үзүүлэлтээс онцолбол:Зээл ба санхүүгийн түрээсийн багц 3,609 тэрбум төгрөгт хүрч, өмнөх улирлаас 8.6%, оны эхнээс 31.2% өссөн нь III улирлын цэвэр хүүгийн орлогыг өмнөх улирлаас 9.0% өсөхөд нөлөөлсөн байна.Активын чанарын үзүүлэлт тогтвортой сайн хэвээр байгаа бөгөөд нийт зээл болон санхүүгийн түрээсийн багцад эзлэх чанаргүй зээлийг 2.2%-д удирдан ажиллаж байна.Харилцах, хадгаламжийн эх үүсвэр 

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)
docs = text_splitter.split_documents(documents=all_data)

print(len(docs))
print(len(all_data))

2832
1203


Find which class prompt belongs to

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bayartsogt/mongolian-gpt2")
tokenizer.pad_token = tokenizer.eos_token

c:\Users\ikhba\Documents\RAG\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import pipeline

generator = pipeline("text-generation", model="Ikhee10/khasbank_three_classifier", tokenizer = tokenizer, device=0, num_beams=5)

In [8]:
import re

def process_generated_text(text):
    # Example regex to extract data in the format you've specified
    matches = re.findall(r'<(.*?)>', text)
    ans = matches[1] if len(matches) > 0 else ''
    if ' ' in ans:
        ans = ans.split(' ')[0].strip()
    return ans
    
def generate_and_process(prompt):
    result = generator(prompt, max_length=32, pad_token_id=tokenizer.eos_token_id)
    generated_text = result[0]['generated_text']

    return process_generated_text(generated_text)

In [6]:
!rd /s /q "../docs/chroma14"  # remove old database files if any

The system cannot find the file specified.
The system cannot find the file specified.
The system cannot find the file specified.
The system cannot find the file specified.
The system cannot find the file specified.
The system cannot find the file specified.
The system cannot find the file specified.


Embeddings

In [ ]:
import openai
import numpy as np

openai.api_key = ''

In [6]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [1]:
from langchain.vectorstores import Chroma

persist_directory = '../docs/chroma14'

In [ ]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory,
    collection_name='v_db',
)

In [10]:
print(vectordb._collection.count())

2832


In [11]:
vectordb.persist()

C:\Users\ikhba\AppData\Local\Temp\ipykernel_36420\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [7]:
# Load from disk
vectordb = Chroma(collection_name='v_db', persist_directory='../docs/chroma14', embedding_function=embedding)

C:\Users\ikhba\AppData\Local\Temp\ipykernel_26208\837188166.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(collection_name='v_db', persist_directory='../docs/chroma14', embedding_function=embedding)


Similarity Search

In [15]:
query = "Баянгол салбарын байршил хаана вэ?"
context = vectordb.similarity_search(query, k=5, filter={"type": "bra"})
for c in context:
    print(c)

page_content='Хан-Уул салбар ATM АТМ \n\nСалбар: 3-р хороо, Чингисийн өргөн чөлөө
Хан-Уул дүүрэг цагаан хаалга \nЦагийн хуваарь: 24 цаг \nУтас: 1800-1888 \n' metadata={'source': 'Хан-Уул салбар ATM', 'type': 'bra'}
page_content='Ханжаргалант тооцооны төв салбар \n\nСалбар: Улиастай сум, Богдын гол баг, "Сэргээгдэх" төв, 1-р давхарт
3 үгт хаяг: /// эмхэтгэл.мэдэрч.ажигч \nЦагийн хуваарь: Даваа - Баасан: 8:30-16:30
Бямба, Ням: Амарна \nУтас: 1800-1888 \n' metadata={'source': 'Ханжаргалант тооцооны төв', 'type': 'bra'}
page_content='Хонгор тооцооны төв салбар \n\nСалбар: Баянхонгор сум, 1-р  баг, "Цэлмэг өглөө" худалдааны төвийн 2-р давхарт
3 үгт хаяг: /// давж.хэдийг.хацартай \nЦагийн хуваарь: Даваа - Баасан: 10:00-18:00
Бямба, Ням: Амарна \nУтас: 1800-1888 \n' metadata={'source': 'Хонгор тооцооны төв', 'type': 'bra'}
page_content='Баянзүрх салбар салбар \n\nСалбар: Баянзүрх дүүрэг, Энх тайвны өргөн чөлөө 14-р хороолол,13 дугаар хороо, 92-А байр 201 тоот
3 үгт хаяг: ///цамц.хатагтай.төрд

In [13]:
query = "Хадгаламж зээл барьцаалсан зээлийн хүү хэд вэ"
context = vectordb.similarity_search(query, k=5, filter={"type":"pro"})
for c in context:
    print(c)

page_content='Хадгаламжийн хайрцаг\n\nХадгаламжийн хайрцаг
Үнэт зүйл хадгалах үйлчилгээ нь танд үнэт зүйлсээ гал, усны аюул болон хулгайд алдахаас сэргийлэх хамгийн хямд зөв арга зам юм. Бид таны эдгээр зүйлсийг хамгийн найдвартайгаар хадгалан хамгаална.
Давуу тал:
Өндөр үнэ цэнэтэй зүйлсдээ санаа зовох зүйлгүй найдвартай хадгална;
Таны хайрцагт юу байгааг зөвхөн та л мэднэ;
Таны хайрцаг гадны нөлөөлөлд тэсвэртэй, тусгай зориулалтын давхар ханаар хуяглагдсан;
Таны эд зүйл хадгалж буй өрөөний хана, хаалга нь халдлагын эсрэг орчин үеийн 24 цагийн дохиолол хамгаалалтын системтэй;
Банкны нийтэд үйлчлэх цагаар та хэдэн удаа ч хайрцагаа ашиглаж болно;
Та өөрт шаардлагатай тооны хайрцгаа хүссэн хугацаагаар эзэмшиж болно;
Хайрцгийг хамтран эзэмшиж, итгэмжлэх боломжтой.
Хураамж, шимтгэл:
Хураамжийн төрөл
1 сарын хураамж /төгрөгөөр/
Хайрцаг эзэмших гэрээ байгуулах
-
Хайрцаг эзэмших гэрээ сунгах
-
Хайрцагны төрөл
Жижиг хайрцаг
10,000
Дунд хайрцаг
12,000
Том хайрцаг
15,000
Том хайрцаг
20,000' meta

In [16]:
query = "Тэтгэврийн зээлд хамрагдах боломжтой хүмүүсийн талаар мэдээлэл өгнө үү"
context = vectordb.similarity_search(query, k=5, filter={"type": "pro"})
for c in context:
    print(c)

page_content='татаж авах
/;
Хуулийн этгээдийн улсын бүртгэлийн гэрчилгээ /нотариатаар батлуулсан хуулбарын хамт/;
Гүйцэтгэх удирдлагын цээж зураг, иргэний үнэмлэхийн хуулбар /зөвхөн анх удаа зээл хүсэгчид хамаатай/;
Үүсгэн байгуулагчдын хооронд байгуулсан гэрээ /нотариатаар батлуулсан хуулбарын хамт/;
Өмнөх 1 жилийн болон сүүлийн улирлын Татварын албанаас баталгаажуулсан тайлан тэнцлийн хуулбар;
Зээлээр санхүүжүүлэх үйл ажиллагаатай холбоотой гэрээ, хэлцлийн хуулбар;
Барьцаалуулах хөрөнгийг өмчлөх эрхийн гэрчилгээ;
Банкны зүгээс шаардлагатай гэж үзсэн бусад нэмэлт материал' metadata={'source': 'https://www.xacbank.mn/product/25', 'title': '25', 'type': 'pro'}
page_content='** Даатгалын үнэлгээг хэрхэн тогтоох нь даатгуулагчийн сонголт байдаг боловч даатгалын тохиолдлын үед хохирлын хэдэн хувийг нөхөн төлөх нь хувь тэнцүүлэгдэх зарчимд суурилдаг тул зах зээлийн үнэлгээтэй тэнцүү хэмжээгээр даатгалын үнэлгээг тогтоохыг даатгагч зөвлөдөг.
Санхүүгийн зохицуулах хороо болон холбогдох мэргэж

Llama-3-70B-Instruct

In [14]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

c:\Users\ikhba\Documents\RAG\.conda\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [24]:
def truncate_context(query, context, max_length=8180):
    template = """Доорх асуултанд хариулаарай. Хариулахад дараах мэдээллийг ашиглана уу. Хэрэв та хариултаа мэдэхгүй бол мэдэхгүй гэж хэлээрэй, хариулт зохиох гэж бүү оролдоорой. Хамгийн ихдээ гурван өгүүлбэр ашигла. Хариултаа аль болох товчхон бичээрэй. Хариултын төгсгөлд "Асуусанд баярлалаа!" гэж үргэлж хэлээрэй.
    {context}
    Асуулт: {query}
    Хариулт: Хариулахад алхам алхмаар бодож үзье."""

    remaining_length = max_length - len(template.lower().format(context="", query=query))
    truncated_context = context[:remaining_length]
    new_template = template.format(context=truncated_context, query=query)
    return new_template

In [16]:
repo_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.1,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

In [25]:
def handle_query(query):
    # Retreival  
    context = mmr_search(query, k=8, fetch_k=20, lambda_param=0.7)
    
    formatted_context = format_context(context.lower())

    prompt = PromptTemplate.from_template(truncate_context(query, context))

    # Generation
    llm_chain = prompt | llm
    response = llm_chain.invoke({"context": context, "query": query})

    return formatted_context, response

In [26]:
# Function to format the context 
def format_context(context): 
    lines = context.split("\n") 
    formatted_lines = [] 
    for line in lines: 
        formatted_lines.append(line[:200]) # Truncate each line to 200 characters 
        
    return "\n".join(formatted_lines)

In [27]:
query = "Банкны хамгийн том түүхэн амжилтууд юу вэ?"

print("Query", "\n-----------------------")
print(query)

context, response = handle_query(query)
print()
print("Retrieval:", "\n-----------------------")
print(context)

# Generation
print()
print("Generation", "\n-----------------------")
print(response)

Query 
-----------------------
Банкны хамгийн том түүхэн амжилтууд юу вэ?

Retrieval: 
-----------------------
https://www.xacbank.mn/article/297: явж байгаатай зүйрлэж болох байх. учир нь, “хас” банк банкны лиценз авсан бусад арилжааны банкууд дотроо хамгийн “залуу”-д тооцогдоно. гэхдээ ололт амжилтын хувьд э
https://www.xacbank.mn/article/267: оюутан танд сургалтын төлбөрөө төлөх хэрэгцээ байна уу ?  та бүхнийг хас банкны  виттана төсөлд хамрагдахыг урьж байна\n\nоюутан танд сургалтын төлбөрөө төлөх хэрэг
https://www.xacbank.mn/article/20th-anniversay: үзүүлэлт хэд дахин өсөж, анх төсөөлөө ч үгүй байсан олон амжилтыг бүтээсэн ч анхны зорилго, үзэл санаандаа үнэнч хэвээр үлдсэн нь бидний 20 жилийн ойн х
https://www.xacbank.mn/article/bankshot: болно.bankshot гэж юу вэ?bankshot гэдэг нь бидний хэлж заншсанаар самбардаж оруулах буюу бөмбөг цагирагны арын самбарт ойн орохыг ойлгоно. гэхдээ үүнд хоёр алх
https://www.xacbank.mn/article/ipo-ceo-interview: л асуудал.он гарсаар дэлхийн томоох

In [29]:
query = "Зээлийн жилийн хүү 8%-тай байж болох уу?"

print("Query", "\n-----------------------")
print(query)

context, response = handle_query(query)
print()
print("Retrieval:", "\n-----------------------")
print(context)

# Generation
print()
print("Generation", "\n-----------------------")
print(response)

Query 
-----------------------
Зээлийн жилийн хүү 8%-тай байж болох уу?

Retrieval: 
-----------------------
https://www.xacbank.mn/product/7: хадгаламж барьцаалсан зээл
хэмжээ
хадгаламжийн 80-85% хүртэл
хугацаа
24 сар
хүү (жилийн)
хадгаламжийн жилийн хүү + /4%-7.2%/\n\nхадгаламж барьцаалсан зээл
харилцагч та гэнэтийн санхүүгийн хэрэгцээгээ шийдвэрлэх гэж байгаа бол “хадгаламж барьцаалсан зээл”-ийг сонгоорой. энэхүү хадгаламж барьцаалсан зээлийг сонгосноор та өөрийн хадгаламж болон хүүгийн хур
бүтээгдэхүүнийн нөхцөл:
олгох валют
хугацаа
дээд хэмжээ
доод
хэмжээ
жилийн хүү
(хадгаламжийн хүү + нэмэгдүүлэх хувь)
зээл олголтын мөнгөн дүн
нэмэгдүүлэх хувь
төгрөг
-
барьцаалж буй мөнгөн хадгаламжийн дансны валютаар
барьцаалж буй хадгаламжийн гэрээний хугацаанаас хэтрэхгүй буюу дээд хугацаа 24 сар
барьцаалж буй мөнгөн хадгаламжийн дансны үлдэгдэл дүнгийн 85% хүртэл
төгрөг - 100,000
0.1-3 сая
+7.20%
-
3.01-50 сая
+6.40%
50.01-100 сая
+5.60%
100.01-500 сая
+4.80%
500 сая-с дээш
+4.00%
цахим банка

In [31]:
query = "Monpay мэдээлэл авъя"

print("Query", "\n-----------------------")
print(query)

context, response = handle_query(query)
print()
print("Retrieval:", "\n-----------------------")
print(context)

# Generation
print()
print("Generation", "\n-----------------------")
print(response)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Query 
-----------------------
Monpay мэдээлэл авъя

Retrieval: 
-----------------------
улаангом тооцооны төв, экспресс банк атм: улаангом тооцооны төв, экспресс банк атм атм \n\nсалбар: увс аймаг, улаангом сум, 3-р баг, хас банк улаангом тооцооны төв  \nцагийн хуваарь: даваа-ням 09:00-2
гандан моми : гандан моми  атм \n\nсалбар: бгд орхон 2-130 \nцагийн хуваарь: даваа-ням \nутас:  \n
дэнжийн 1000 сүүн дэлгүүр: дэнжийн 1000 сүүн дэлгүүр атм \n\nсалбар: 10-р хороо; шинэчлэл хороолол \nцагийн хуваарь:  09-22 цаг \nутас: 1800-1888 \n
100 айл салбар atm: 100 айл салбар atm атм \n\nсалбар: 10-р хороо, цагдаагийн гудамж
/100 айл/ \nцагийн хуваарь: даваа гарагт амарна.
бусад өдөр 10-19 цаг \nутас: 1800-1888 \n
бөхийн өргөө баянзүрх хүнсний зах атм: бөхийн өргөө баянзүрх хүнсний зах атм атм \n\nсалбар: 6-р хороо, бөхийн өргөө баянзүрх хүнсний зах \nцагийн хуваарь: даваа-ням 24/7 цаг \nутас: 1800-1888 \n
их-орд лоби crm: их-орд лоби crm атм \n\nсалбар: 14-р хороо, намъянжүгийн гудамж, их орд х

In [44]:
import gradio as gr

In [45]:
# Create Gradio interface 
iface = gr.Interface( 
    fn=handle_query, 
    inputs="text", 
    outputs=["text", "text"], 
    examples=[["Хамгийн өндөр цалинтай ажил юу вэ?"]], 
    title="Khas Bank Q&A Interface", 
    description="Асуултаа оруулна уу?"
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [46]:
iface.close()

Closing server running on port: 7860
